In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("playtennis.csv")
data.head()

,Day,Outlook,Temperature,Humidity,Wind,Play Tennis
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No
2,3,Overcast,Hot,High,Weak,Yes
3,4,Rain,Mild,High,Weak,Yes
4,5,Rain,Cool,Normal,Weak,Yes


In [3]:
def getID3Tree():
    passdef getID3Tree():
    pass

SyntaxError: invalid syntax (<ipython-input-3-9e1ab5bc3f7d>, line 2)

In [7]:
def predictByTree():
    pass

In [8]:
class Node:
    def __init__(self, label):
        self.label = label

In [110]:
import re
import math
from collections import deque

# x is examples in training set
# y is set of attributes
# label is target attributes
# Node is a class which has properties values, childs, and next
# root is top node in the decision tree

class Node(object):
    def __init__(self):
        self.value = None
        self.next = None
        self.childs = None

# Simple class of Decision Tree
# Aimed for who want to learn Decision Tree, so it is not optimized
class DecisionTree(object):
    def __init__(self, sample, attributes, labels):
        self.sample = sample
        self.attributes = attributes
        self.labels = labels
        self.labelCodes = None
        self.labelCodesCount = None
        self.initLabelCodes()
        # print(self.labelCodes)
        self.root = None
        self.entropy = self.getEntropy([x for x in range(len(self.labels))])

    def initLabelCodes(self):
        self.labelCodes = []
        self.labelCodesCount = []
        for l in self.labels:
            if l not in self.labelCodes:
                self.labelCodes.append(l)
                self.labelCodesCount.append(0)
            self.labelCodesCount[self.labelCodes.index(l)] += 1
        print(self.labelCodes)
        print(self.labelCodesCount)

    def getLabelCodeId(self, sampleId):
        return self.labelCodes.index(self.labels[sampleId])

    def getAttributeValues(self, sampleIds, attributeId):
        vals = []
        for sid in sampleIds:
            val = self.sample[sid][attributeId]
            if val not in vals:
                vals.append(val)
        # print(vals)
        return vals

    def getEntropy(self, sampleIds):
        entropy = 0
        labelCount = [0] * len(self.labelCodes)
        for sid in sampleIds:
            labelCount[self.getLabelCodeId(sid)] += 1
        # print("-ge", labelCount)
        for lv in labelCount:
            # print(lv)
            if lv != 0:
                entropy += -lv/len(sampleIds) * math.log(lv/len(sampleIds), 2)
            else:
                entropy += 0
        return entropy

    def getDominantLabel(self, sampleIds):
        print("=======")
        print(sampleIds)
        labelCodesCount = [0] * len(self.labelCodes)
        for sid in sampleIds:
            labelCodesCount[self.labelCodes.index(self.labels[sid])] += 1
        return self.labelCodes[labelCodesCount.index(max(labelCodesCount))]

    def getInformationGain(self, sampleIds, attributeId):
        gain = self.getEntropy(sampleIds)
        attributeVals = []
        attributeValsCount = []
        attributeValsIds = []
        for sid in sampleIds:
            val = self.sample[sid][attributeId]
            if val not in attributeVals:
                attributeVals.append(val)
                attributeValsCount.append(0)
                attributeValsIds.append([])
            vid = attributeVals.index(val)
            attributeValsCount[vid] += 1
            attributeValsIds[vid].append(sid)
        # print("-gig", self.attributes[attributeId])
        for vc, vids in zip(attributeValsCount, attributeValsIds):
            # print("-gig", vids)
            gain -= vc/len(sampleIds) * self.getEntropy(vids)
            print("000000000")
            print(len(sampleIds))
        return gain

    def getAttributeMaxInformationGain(self, sampleIds, attributeIds):
        attributesEntropy = [0] * len(attributeIds)
        for i, attId in zip(range(len(attributeIds)), attributeIds):
            attributesEntropy[i] = self.getInformationGain(sampleIds, attId)
        maxId = attributeIds[attributesEntropy.index(max(attributesEntropy))]
        return self.attributes[maxId], maxId

    def isSingleLabeled(self, sampleIds):
        label = self.labels[sampleIds[0]]
        for sid in sampleIds:
            if self.labels[sid] != label:
                return False
        return True

    def getLabel(self, sampleId):
        return self.labels[sampleId]

    def id3(self):
        sampleIds = [x for x in range(len(self.sample))]
        attributeIds = [x for x in range(len(self.attributes))]
        self.root = self.id3Recv(sampleIds, attributeIds, self.root)

    def id3Recv(self, sampleIds, attributeIds, root):
        print("++++++++++")
        print(sampleIds)
        print(attributeIds)
        root = Node() # Initialize current root
        if self.isSingleLabeled(sampleIds):
            root.value = self.labels[sampleIds[0]]
            return root
        # print(attributeIds)
        if len(attributeIds) == 0:
            root.value = self.getDominantLabel(sampleIds)
            return root
        bestAttrName, bestAttrId = self.getAttributeMaxInformationGain(
            sampleIds, attributeIds)
        # print(bestAttrName)
        root.value = bestAttrName
        root.childs = []  # Create list of children
        for value in self.getAttributeValues(sampleIds, bestAttrId):
            # print(value)
            child = Node()
            child.value = value
            root.childs.append(child)  # Append new child node to current
                                       # root
            childSampleIds = []
            for sid in sampleIds:
                if self.sample[sid][bestAttrId] == value:
                    childSampleIds.append(sid)
            if len(childSampleIds) == 0:
                child.next = self.getDominantLabel(sampleIds)
            else:
                # print(bestAttrName, bestAttrId)
                # print(attributeIds)
                if len(attributeIds) > 0 and bestAttrId in attributeIds:
                    toRemove = attributeIds.index(bestAttrId)
                    attributeIds.pop(toRemove)
                child.next = self.id3Recv(
                    childSampleIds, attributeIds, child.next)
        return root
    
    def printTree(self):
            if self.root:
                roots = deque()
                roots.append(self.root)
                while len(roots) > 0:
                    root = roots.popleft()
                    print(root.value)
                    if root.childs:
                        for child in root.childs:
                            print('({})'.format(child.value))
                            roots.append(child.next)
                    elif root.next:
                        print(root.next)

In [111]:
import re
f = open('playtennis.csv')
attributes = f.readline().split(',')
attributes = attributes[1:len(attributes)-1]
print(attributes)
sample = f.readlines()
f.close()
for i in range(len(sample)):
    sample[i] = re.sub('\d+,', '', sample[i])
    sample[i] = sample[i].strip().split(',')
labels = []
for s in sample:
    labels.append(s.pop())
print(sample)
print(labels)

['Outlook', 'Temperature', 'Humidity', 'Wind']
[['Sunny', 'Hot', 'High', 'Weak'], ['Sunny', 'Hot', 'High', 'Strong'], ['Overcast', 'Hot', 'High', 'Weak'], ['Rain', 'Mild', 'High', 'Weak'], ['Rain', 'Cool', 'Normal', 'Weak'], ['Rain', 'Cool', 'Normal', 'Strong'], ['Overcast', 'Cool', 'Normal', 'Strong'], ['Sunny', 'Mild', 'High', 'Weak'], ['Sunny', 'Cool', 'Normal', 'Weak'], ['Rain', 'Mild', 'Normal', 'Weak'], ['Sunny', 'Mild', 'Normal', 'Strong'], ['Overcast', 'Mild', 'High', 'Strong'], ['Overcast', 'Hot', 'Normal', 'Weak'], ['Rain', 'Mild', 'High', 'Strong']]
['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']


In [112]:
decisionTree = DecisionTree(sample, attributes, labels)
print("System entropy {}".format(decisionTree.entropy))
decisionTree.id3()
decisionTree.printTree()

['No', 'Yes']
[5, 9]
System entropy 0.9402859586706309
++++++++++
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[0, 1, 2, 3]
000000000
14
000000000
14
000000000
14
000000000
14
000000000
14
000000000
14
000000000
14
000000000
14
000000000
14
000000000
14
++++++++++
[0, 1, 7, 8, 10]
[1, 2, 3]
000000000
5
000000000
5
000000000
5
000000000
5
000000000
5
000000000
5
000000000
5
++++++++++
[0, 1, 7]
[1, 3]
++++++++++
[8, 10]
[1, 3]
++++++++++
[2, 6, 11, 12]
[1, 3]
++++++++++
[3, 4, 5, 9, 13]
[1, 3]
000000000
5
000000000
5
000000000
5
000000000
5
++++++++++
[3, 4, 9]
[1]
++++++++++
[5, 13]
[1]
Outlook
(Sunny)
(Overcast)
(Rain)
Humidity
(High)
(Normal)
Yes
Wind
(Weak)
(Strong)
No
Yes
Yes
No


# Алгоритм ID3

In [53]:
TARGET_COLUMN = 'Play Tennis'

class Node(object):
    def __init__(self):
        self.value = None
        self.next = None
        self.childs = None

In [51]:
def getAttributeMaxInformationGain(self, sampleIds, attributeIds):
    attributesEntropy = [0] * len(attributeIds)
    for i, attId in zip(range(len(attributeIds)), attributeIds):
        attributesEntropy[i] = self.getInformationGain(sampleIds, attId)
    maxId = attributeIds[attributesEntropy.index(max(attributesEntropy))]
    return self.attributes[maxId], maxId

In [ ]:
def getEntropy(counts_lst):
    """
    Вычисление энтропии по списку с количествами различных классов
    """
    entropy = 0
    for cnt in counts_lst:
        if cnt != 0:
            entropy += -cnt/len(counts_lst) * math.log(cnt/len(counts_lst), 2)

    return entropy

In [ ]:
def getInformationGain(X_train, y_train, attr_name):
    """
    Вычисление information gain для конкретного атрибута
    """
    gain = self.getEntropy(sampleIds)
    attributeVals = []
    attributeValsCount = []
    attributeValsIds = []
    for sid in sampleIds:
        val = self.sample[sid][attributeId]
        if val not in attributeVals:
            attributeVals.append(val)
            attributeValsCount.append(0)
            attributeValsIds.append([])
        vid = attributeVals.index(val)
        attributeValsCount[vid] += 1
        attributeValsIds[vid].append(sid)
    # print("-gig", self.attributes[attributeId])
    for vc, vids in zip(attributeValsCount, attributeValsIds):
        # print("-gig", vids)
        gain -= vc/len(sampleIds) * self.getEntropy(vids)
        print("000000000")
        print(len(sampleIds))
    return gain

In [58]:
def getID3Tree(X_train, y_train, attributes, target_classes, root):
    """ 
    Функция генерации ID3 дерева 
    """
    root = Node()
    if y_train.nunique() == 1:
        root.value = y_train[0]
        return root
    if len(attributes) == 0:
        root.value = y_train.value_counts().idxmax()
        return root
    bestAttrName, bestAttrId = self.getAttributeMaxInformationGain(
    sampleIds, attributeIds)
    # print(bestAttrName)
    root.value = bestAttrName

In [59]:
data = pd.read_csv("playtennis.csv")
data.drop('Day', axis=1, inplace=True)
data.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [60]:
X_train = data.loc[:, data.columns != TARGET_COLUMN]
y_train = data[TARGET_COLUMN]
attributes = X_train.columns
y_train.nunique()

2

In [61]:
root = getID3Tree(X_train, y_train, attributes, y_train.unique(), None)

NameError: name 'self' is not defined

array(['No', 'Yes'], dtype=object)

In [48]:
y_train.value_counts().idxmax()

'Yes'

In [46]:
counts

'Yes'

In [47]:
y_train.value_counts()

Yes    9
No     5
Name: Play Tennis, dtype: int64